## Распознавание Именованных Сущностей с помощью библиотеки DeepPavlov

Задачей Распознавания Именованных Сущностей (РИС) называется извлечение из текста таких объектов как имена, названия организаций, названия географических объектов. Данная задача как правило является компонентом в более крупной системе. Например, в диалоговой системе РИС может быть использован для выделения имени собеседника. В библиотеке [DeepPavlov](https://github.com/deepmipt/DeepPavlov) есть ряд моделей которые решают данную задачу. В данном notebook-е мы рассмотрим две модели решающие задачу РИС на русском языке: [BERT](https://arxiv.org/pdf/1810.04805.pdf), показывающий на данный момент наилучшее качество, и [Bi-LSTM-CRF](https://arxiv.org/pdf/1603.01360.pdf) - уступающий по метрикам, однако более быстрый baseline.

## Постановка задачи

Задача РИС может быть поставлена следующим образом: для заданной последовательность слов (токенов) предсказать последовательность тагов (меток). Каждому входному токену сопоставляется таг из заданного множества тагов. Пример:

    Алиса  в  Стране чудес
     PER   O   LOC    LOC

здесь PER - персона, LOC - локация. Однко, представленная разметка не позволяет разделять подряд идущие сущности. Для разделения таких сущностей используют префиксы B и I перед каждым тагом кроме O. Префикс B обозначает начало сущности, а I - продолжение. Тогда для примера выше будет следующая разметка:

    Алиса  в  Стране чудес
    B-PER  O  B-LOC  I-LOC

Разметка с префиксами B и O - наиболее распространённый способ разметки данных. Данный тип разметки часто называют BIO или IOB.

## Dataset

Рассматриваемые в данном notebook-е модели были обучены на датасете [1]. Данный датасет содержит 1000 новостей в которых размечены персоны (PER), локации (LOC) и организации (ORG). В силу того, что обучающая выборка содержит только новостные данные смена типов распознаваемых текстов может существенно отразиться на качестве работы системы. Например, при использовании модели обученной на новостях переход к распознавания диалогов службы поддрежки может существенно снизить качество работы системы.

1. Mozharova V., Loukachevitch N., Two-stage approach in Russian named entity recognition // International FRUCT Conference on Intelligence, Social Media and Web, ISMW FRUCT 2016. Saint-Petersburg; Russian Federation, DOI 10.1109/FRUCT.2016.7584769

## Установка библиотеки

In [1]:
!pip install -q deeppavlov

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.2.2 requires fsspec, which is not installed.


## Установка зависимостей, спецефичных для модели

In [2]:
!python -m deeppavlov install ner_rus_bert

Ignoring transformers: markers 'python_version < "3.8"' don't match your environment
     ---------------------------------------- 0.0/113.6 kB ? eta -:--:--
     --- ------------------------------------ 10.2/113.6 kB ? eta -:--:--
     ---------- -------------------------- 30.7/113.6 kB 330.3 kB/s eta 0:00:01
     ------------- ----------------------- 41.0/113.6 kB 326.8 kB/s eta 0:00:01
     ------------------------------ ------ 92.2/113.6 kB 585.1 kB/s eta 0:00:01
     ------------------------------------ 113.6/113.6 kB 662.5 kB/s eta 0:00:00
  Using cached huggingface_hub-0.22.2-py3-none-any.whl.metadata (12 kB)
  Using cached safetensors-0.4.2-cp311-none-win_amd64.whl.metadata (3.9 kB)
  Using cached fsspec-2024.3.1-py3-none-any.whl.metadata (6.8 kB)
   ---------------------------------------- 0.0/7.2 MB ? eta -:--:--
    --------------------------------------- 0.1/7.2 MB 3.6 MB/s eta 0:00:02
    --------------------------------------- 0.2/7.2 MB 1.8 MB/s eta 0:00:04
   -- -------

ERROR: Could not find a version that satisfies the requirement torch<1.14.0,>=1.6.0 (from versions: 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2)
ERROR: No matching distribution found for torch<1.14.0,>=1.6.0
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\totmi\anaconda\envs\Diplom\Lib\site-packages\deeppavlov\__main__.py", line 4, in <module>
    main()
  File "C:\Users\totmi\anaconda\envs\Diplom\Lib\site-packages\deeppavlov\deep.py", line 62, in main
    install_from_config(pipeline_config_path)
  File "C:\Users\totmi\anaconda\envs\Diplom\Lib\site-packages\deeppavlov\utils\pip_wrapper\pip_wrapper.py", line 71, in install_from_config
    install(r)
  File "C:\Users\totmi\anaconda\envs\Diplom\Lib\site-packages\deeppavlov\utils\pip_wrapper\pip_wrapper.py", line 36, in install
    result = subprocess.check_call([sys.executable, '-m', 'pip', 'install',
             ^^^^^

## Использование моделей

### BERT

BERT - сеть архитектуры Transformer предобученная на задаче Masked Language Modelling (MLM). Модель осуществляющая РИС использует [RuBERT](https://arxiv.org/abs/1905.07213) предобученный на русском языке.

In [3]:
from deeppavlov import configs, build_model

config_path = configs.ner.ner_rus_bert

ner = build_model(config_path, download=True)

2024-04-04 18:55:26.175 INFO in 'deeppavlov.core.data.utils'['utils'] at line 97: Downloading from http://files.deeppavlov.ai/v1/ner/ner_rus_bert_torch_new.tar.gz to C:\Users\totmi\.deeppavlov\models\ner_rus_bert_torch_new.tar.gz
100%|██████████| 1.44G/1.44G [02:21<00:00, 10.2MB/s] 
2024-04-04 18:57:48.567 INFO in 'deeppavlov.core.data.utils'['utils'] at line 284: Extracting C:\Users\totmi\.deeppavlov\models\ner_rus_bert_torch_new.tar.gz archive into C:\Users\totmi\.deeppavlov\models\ner_rus_bert_torch
C:\Users\totmi\anaconda\envs\Diplom\Lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weigh

## Обработка входных данных

In [23]:
import numpy as np 
import pandas as pd 
import torch
from torch import cuda
from tqdm import tqdm

In [24]:
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [33]:
data = pd.read_csv('Train_Lenta.csv', encoding='utf-8-sig', sep=';')
data.head()

,id,text
0,800000,Самый низкий показатель - среди жителей Японии...
1,800001,В самих США положительно ответили более 20 про...
2,800002,"Аналитики MPAA считают, что распространение пи..."
3,800003,При недостаточно высокой скорости соединения с...
4,800004,Лидерство Южной Кореи в опросе обусловлено име...


In [34]:
data.count()

id      1009468
text    1009467
dtype: int64

In [35]:
# Замена пропущенных значений или значений других типов на пустые строки
data['text'] = data['text'].fillna('').astype(str)

In [36]:
# Очистка текста от лишних символов
data['text'] = data['text'].str.strip()

# Применение модели NER к текстам из колонки 'text' и сохранение результатов в новую колонку 'tag'
tags_list = []
for sentence in tqdm(data['text'], desc="Processing sentences"):
    if isinstance(sentence, str) and sentence.strip():  # Проверяем, является ли значение текстом и не пустым
        tokens, tags = ner([sentence])
        tags_list.append(tags[0])
    else:
        tags_list.append([])  # Добавляем пустой список тегов

data['tag'] = tags_list

Processing sentences: 100%|██████████| 1009468/1009468 [2:39:43<00:00, 105.33it/s] 


In [38]:
# Сохранение обновленных данных в новый CSV-файл
data.to_csv('Train_Lenta_NER.csv', encoding='utf-8-sig', sep=';', index=False)

In [39]:
data = pd.read_csv('Train_Lenta_NER.csv', encoding='utf-8-sig', sep=';')
data.head()

,id,text,tag
0,800000,Самый низкий показатель - среди жителей Японии...,"['O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', '..."
1,800001,В самих США положительно ответили более 20 про...,"['O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', '..."
2,800002,"Аналитики MPAA считают, что распространение пи...","['O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', '..."
3,800003,При недостаточно высокой скорости соединения с...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
4,800004,Лидерство Южной Кореи в опросе обусловлено име...,"['O', 'B-LOC', 'I-LOC', 'O', 'O', 'O', 'O', 'O..."


In [ ]:
class T5MTL(torch.nn.Module):
    def __init__(self, model_params: dict, task_type: str) -> None:
        super(T5MTL, self).__init__()
        self.task_type = task_type
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.tokenizer = T5Tokenizer.from_pretrained(model_params["MODEL"])
        self.encoder = T5Model.from_pretrained(model_params["MODEL"]).to(self.device)
        self.decoder = T5ForConditionalGeneration.from_pretrained(model_params["MODEL"]).to(self.device)
        self.classifier = torch.nn.Linear(self.encoder.config.d_model, model_params["NUM_CLASSES"]).to(self.device)

    def forward(self, input_ids, attention_mask, decoder_input_ids=None, labels=None):
        # Encoding phase
        encoder_outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        encoder_last_hidden_state = encoder_outputs.last_hidden_state

        if self.task_type in ['paraphrase_task', 'qg_task', 'sum_task', 'titlegen_task']:
            # Decoding phase for generation
            if decoder_input_ids is not None:
                outputs = self.decoder(input_ids=decoder_input_ids, encoder_hidden_states=encoder_last_hidden_state, labels=labels)
            else:
                outputs = self.decoder.generate(input_ids=None, encoder_hidden_states=encoder_last_hidden_state)
        elif self.task_type in ['ner_task', 'sts_task', 'nli_task', 'sentiment_task']:
            # Classification phase
            outputs = self.classify(encoder_last_hidden_state)

        return outputs